# Rheostat Biocrnpyler Attempt 3
## Allow for List Inputs
5.21.2020 <br>
<font color = 'red'>
    fix enzyme_name issue<br>
    is ODE int a problem? <br>
  <font color = 'pink'>
    more efficient way to incorporate list inputs?<br>
    make a new mechanism for just enzyme? - not necessary
    </font>

In [1]:
# import cell
import bokeh.io
import bokeh.plotting
from bokeh.layouts import row
from bokeh.layouts import column
#import bokeh_catplot
bokeh.io.output_notebook()

from biocrnpyler.mechanism import Mechanism
from biocrnpyler.component import Component
from biocrnpyler import Mixture
from biocrnpyler.chemical_reaction_network import Species, Reaction, ComplexSpecies, ChemicalReactionNetwork
import numpy as np

import warnings
warnings.filterwarnings('ignore')

Loading BokehJS ...

## Define functions

In [2]:
def check_type(item, material_type_str): # call set_species
    if isinstance(item, Species):
        item_ret = item
    elif isinstance(item, str):
        item_ret = Species(name = item, material_type = material_type_str)
    elif isinstance(item, Component) and item.get_species() != None:
        item_ret = item.get_species()
    else:
        raise ValueError( item, "parameter must be a string, a Component with defined get_species(), or a chemical_reaction_network.species")
        
    return item_ret

## Mechanism: FuelMichaelisMenten

In [3]:
class FuelMichaelisMenten(Mechanism):
    def __init__(self, name, type = 'catalysis', **keywords):
        
        
   
        
        Mechanism.__init__(self = self, name = name, mechanism_type = type, **keywords)

    def update_species(self, enzyme, fuel_list, substrate_list, product_list, waste_list): 
        
        self.enzyme = check_type(enzyme, 'enzyme')
        
        species = [self.enzyme]
        comp1_list = [self.enzyme]
        comp2_list = [self.enzyme]
        
        for f in fuel_list:
            species.append(f)
            comp1_list.append(f)

        for s in substrate_list:
            species.append(s)
            comp1_list.append(s)
            
        for p in product_list:
            species.append(p)
            comp2_list.append(p)
            
        for w in waste_list:
            species.append(w)
            comp2_list.append(w)
               
        
        species += [ComplexSpecies(comp1_list)]
        species += [ComplexSpecies(comp2_list)]
        return species
    
    def update_reactions(self, enzyme, fuel_list, substrate_list, product_list, waste_list, k_cat, component, part_id = None):

        k_bf = 22.68
        
        #k_bf = component.get_parameter('k_bf', part_id = part_id, mechanism = self)
        
        k_br = 2.268
        k_uf = 24
        k_ur = 2.4
       # k_cat = 10
       # k_atp_use = 0.5
       # print(enzyme)
        self.enzyme = check_type(enzyme, 'enzyme')
        
        
        comp1_list = [self.enzyme]
        comp2_list = [self.enzyme]


        for f in fuel_list:
            comp1_list.append(f)
            
        for s in substrate_list:
            comp1_list.append(s)

        for p in product_list:
            comp2_list.append(p)
            
        for w in waste_list:
            comp2_list.append(w)
        
        comp1 = ComplexSpecies(comp1_list)
        comp2 = ComplexSpecies(comp2_list)
        
        binding_rxn = Reaction(inputs = comp1_list, outputs=[comp1], k = k_bf, k_rev = k_br)
        cat_rxn = Reaction(inputs = [comp1], outputs = [comp2], k = k_cat)
        unbinding_rxn = Reaction(inputs = [comp2], outputs = comp2_list, k=k_uf, k_rev = k_ur)
        
        return [binding_rxn, cat_rxn, unbinding_rxn]

## Component: Enzyme

In [4]:
class Enzyme(Component):
    def __init__(self, enzyme_name, substrate, fuel, product, waste, k_cat = 10, **keywords):
      
        # ENZYME NAME
        # self.enzyme = self.set_species()
        self.enzyme = check_type(enzyme_name, 'enzyme')
    
        # SUBSTRATE
        self.substrate_list = []
        for s in substrate:
            self.substrate_list.append(self.set_species(s, material_type = 'molecule'))
            
#         for s in substrate:
#             setattr(self, f'{s}', check_type(s, 'molecule'))
#             substrate_list.append(getattr(self, f'{s}'))
            
#         self.substrate_list = substrate_list
        
        
        # FUEL
        
        self.fuel_list = []
        for f in fuel:
            self.fuel_list.append(self.set_species(f, material_type ='metabolite'))
        
        
        
        # PRODUCT
        self.product_list = []
        for p in product:
            self.product_list.append(self.set_species(p, material_type = 'molecule'))
          
            
        # WASTE
        self.waste_list = []
        for w in waste:
            self.waste_list.append(self.set_species(w, material_type = 'metabolite'))
        
        
        self.k_cat = k_cat
            
        
      
        Component.__init__(self = self, name = enzyme_name, **keywords)
        
    def update_species(self):
        mech_cat = self.mechanisms['catalysis']
        
            
        
        return mech_cat.update_species(self.enzyme, self.fuel_list, self.substrate_list, self.product_list, self.waste_list) 
                                                                                           
    
    def update_reactions(self):
        mech_cat = self.mechanisms['catalysis']

        
        return mech_cat.update_reactions(self.enzyme, self.fuel_list, self.substrate_list, self.product_list, self.waste_list,self.k_cat,
                                             component = self, part_id = self.name) # for parameters


## Mixture: EnergyTxTl

In [5]:
class EnergyTxTl(Mixture):
    def __init__(self, name="",**keywords): 
        

        mech_cat = FuelMichaelisMenten('catalysis')
        
        default_mechanisms = {
            mech_cat.mechanism_type:mech_cat
        }
        
        #species = [self.enzyme]
        Mixture.__init__(self, name = name, default_mechanisms=default_mechanisms, **keywords) 
        #  parameter_file = 'parameters.txt'
    

## Run simulation


In [67]:
# parameter dictionary (choose one, dont mix)
# parameters = {("catalysis","enzyme1", "k_bf"):1.0, 
#               ("default_gene_expression","ReporterBLAH", "ku"):.01,
#               ("default_gene_expression","ReporterBLAH", "kb"):100.0 }


E1 = Enzyme(enzyme_name = "enzyme1", substrate = ['glucose'],
            fuel = ['atp', 'atp'],product = ['f16p'], waste = ['adp','adp'])

E2 = Enzyme(enzyme_name = 'enzyme2', substrate = ['f16p'], fuel = ['adp', 'adp'],
           product = ['isobutanol'], waste = ['atp', 'atp'])

E3 = Enzyme(enzyme_name = 'enzyme3', substrate = ['f16p'], fuel = ['adp', 'adp', 'adp', 'pi'], product = ['isobutanol'],
           waste = ['atp', 'atp', 'atp'])

E4 = Enzyme(enzyme_name = 'enzyme4', substrate = [], fuel = ['atp'], product = [], waste = ['adp', 'pi'], k_cat = 0.5)

myMixture = EnergyTxTl(components = [E1,E2,E3,E4])
CRN = myMixture.compile_crn()

myMixture_atp = EnergyTxTl(components = [E4])
CRN_atp = myMixture_atp.compile_crn()

print(CRN.pretty_print(show_rates = False, show_attributes = False, show_materials = False))
print('\n ATP', CRN_atp.pretty_print(show_rates = False))

Species (18) = {0. enzyme[enzyme1], 1. metabolite[atp], 2. molecule[glucose], 3. molecule[f16p], 4. metabolite[adp], 5. complex[enzyme[enzyme1]:2x_metabolite[atp]:molecule[glucose]], 6. complex[enzyme[enzyme1]:2x_metabolite[adp]:molecule[f16p]], 7. enzyme[enzyme2], 8. molecule[isobutanol], 9. complex[enzyme[enzyme2]:2x_metabolite[adp]:molecule[f16p]], 10. complex[enzyme[enzyme2]:2x_metabolite[atp]:molecule[isobutanol]], 11. enzyme[enzyme3], 12. metabolite[pi], 13. complex[enzyme[enzyme3]:3x_metabolite[adp]:metabolite[pi]:molecule[f16p]], 14. complex[enzyme[enzyme3]:3x_metabolite[atp]:molecule[isobutanol]], 15. enzyme[enzyme4], 16. complex[enzyme[enzyme4]:metabolite[atp]], 17. complex[enzyme[enzyme4]:metabolite[adp]:metabolite[pi]]}
Reactions (12) = [
0. enzyme[enzyme1] + 2 metabolite[atp] + molecule[glucose] <--> complex[enzyme[enzyme1]:2x_metabolite[atp]:molecule[glucose]]        
1. complex[enzyme[enzyme1]:2x_metabolite[atp]:molecule[glucose]] --> complex[enzyme[enzyme1]:2x_metabolit

/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/biocrnpyler-0.1-py3.7.egg/biocrnpyler/component.py:11: UserWarning: get_species() not defined for component {self.name}, None returned.
/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/biocrnpyler-0.1-py3.7.egg/biocrnpyler/component.py:11: UserWarning: get_species() not defined for component {self.name}, None returned.


In [68]:
CRN.write_sbml_file("CRN.sbml")
CRN_atp.write_sbml_file("CRN_atp.sbml")
timepoints = np.linspace(0,72,100)
e = 0.15
e4 = 3.4
atp = 15
x0 = {'molecule_glucose':15,
      'metabolite_atp': atp,
      "enzyme_enzyme1":e,
      "enzyme_enzyme2":e,
      "enzyme_enzyme3":e,
      "enzyme_enzyme4":e4
}
     
x0_atp = {'metabolite_atp':atp,
          "enzyme_enzyme4":e4,

    }

re = CRN.simulate_with_bioscrape(timepoints, initial_condition_dict = x0)
re_atp = CRN_atp.simulate_with_bioscrape(timepoints, initial_condition_dict = x0_atp)

In [69]:
# re, me= CRN.simulate_with_bioscrape_via_sbml(timepoints, initial_condition_dict = x0, file = 'CRN.sbml')
# re_atp,me_atp = CRN_atp.simulate_with_bioscrape_via_sbml(timepoints, initial_condition_dict = x0_atp, file = 'CRN_atp.sbml')



In [70]:
#colors = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3','#a6d854','#ffd92f']
colors=['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02', '#a6761d']
p1 = bokeh.plotting.figure(width =450, height = 250, title = 'Dynamics rxn 1,2,3', x_axis_label = 'time',
                          y_axis_label = 'concentration')
p1.line(timepoints, re['metabolite_atp'], legend_label = 'atp', line_width = 2, color = colors[0])
p1.line(timepoints, re['molecule_glucose'], legend_label = 'glucose', line_width = 2, color = colors[1])
p1.line(timepoints, re['enzyme_enzyme1'], legend_label = 'enzyme1', line_width = 2, color = colors[2])
p1.line(timepoints, re['metabolite_adp'], legend_label = 'adp', line_width = 2, color = colors[3])
p1.line(timepoints, re['molecule_f16p'], legend_label = 'f16p', line_width = 2, color = colors[4])
p1.line(timepoints, re['molecule_isobutanol'], legend_label = 'isobutanol', line_width = 2, color = colors[5])
p1.line(timepoints, re['metabolite_pi'], legend_label = 'pi', line_width = 2, color = colors[6])

p1.legend.click_policy="hide"

p2 = bokeh.plotting.figure(width =450, height = 250, title = 'Dynamics rxn atp leak only', x_axis_label = 'time',
                          y_axis_label = 'concentration', y_range = p1.y_range)
p2.line(timepoints, re_atp['metabolite_atp'], legend_label = 'atp', line_width = 2, color = colors[0])
p2.line(timepoints, re_atp['metabolite_adp'], legend_label = 'adp', line_width = 2, color = colors[3])
p2.line(timepoints, re_atp['enzyme_enzyme4'], legend_label = 'enzyme4',line_width = 2, color = colors[2])
p2.legend.click_policy="hide"

p3 = bokeh.plotting.figure(width =450, height = 250, title = 'Dynamics rxn atp leak for both', x_axis_label = 'time',
                          y_axis_label = 'concentration', y_range = p1.y_range)

p3.line(timepoints, re_atp['metabolite_atp'], legend_label = 'atp_ONLY', line_width = 2, color = colors[0])
p3.line(timepoints, re['metabolite_atp'], legend_label = 'atp_rheostat', line_width = 2, color = colors[1])




bokeh.io.show(row(p1,p2))
bokeh.io.show(p3)

Use rheostat_7_biocrnpyler_compare.ipynb to compare<br>
**PARAMETER SEARCH**

In [29]:
re

,enzyme_enzyme1,metabolite_atp,molecule_glucose,molecule_f16p,metabolite_adp,complex_enzyme_enzyme1_2x_metabolite_atp_molecule_glucose,complex_enzyme_enzyme1_2x_metabolite_adp_molecule_f16p,enzyme_enzyme2,molecule_isobutanol,complex_enzyme_enzyme2_2x_metabolite_adp_molecule_f16p,complex_enzyme_enzyme2_2x_metabolite_atp_molecule_isobutanol,enzyme_enzyme3,metabolite_pi,complex_enzyme_enzyme3_3x_metabolite_adp_metabolite_pi_molecule_f16p,complex_enzyme_enzyme3_3x_metabolite_atp_molecule_isobutanol,enzyme_enzyme4,complex_enzyme_enzyme4_metabolite_atp,complex_enzyme_enzyme4_metabolite_adp,time
0,0.150000,1.500000e+01,15.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.150000,0.000000,0.000000e+00,0.000000e+00,0.150000,0.0,0.0,0.000000e+00,3.400000,0.000000e+00,0.000000,0.000000
1,0.000440,9.177646e+00,14.134996,4.112465e-01,1.904983,1.055477e-01,4.401224e-02,0.048575,0.183136,6.679372e-02,3.463091e-02,0.130363,0.0,0.0,1.963671e-02,0.043508,3.280353e+00,0.076139,0.707071
2,0.000566,7.516116e+00,13.389328,6.009591e-01,3.431654,1.054014e-01,4.403214e-02,0.021933,0.682206,8.264053e-02,4.542657e-02,0.099993,0.0,0.0,5.000658e-02,0.052791,3.261763e+00,0.085446,1.414141
3,0.000741,6.070039e+00,12.645022,7.310204e-01,4.828412,1.051775e-01,4.408148e-02,0.013687,1.273318,8.878233e-02,4.753109e-02,0.084933,0.0,0.0,6.506665e-02,0.064796,3.237335e+00,0.097869,2.121212
4,0.001003,4.747104e+00,11.902862,8.277371e-01,6.142793,1.048198e-01,4.417727e-02,0.009886,1.909069,9.256140e-02,4.755272e-02,0.078779,0.0,0.0,7.122065e-02,0.081781,3.202539e+00,0.115680,2.828283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.150000,9.913095e-10,2.921987,2.749097e-10,13.073549,8.789984e-10,4.562546e-10,0.150000,12.078013,9.361217e-10,5.230784e-10,0.150000,0.0,0.0,1.792161e-10,1.473549,1.453036e-08,1.926451,67.171717
96,0.150000,8.033995e-10,2.921987,2.144805e-10,13.073549,6.715358e-10,3.466387e-10,0.150000,12.078013,8.033056e-10,4.503267e-10,0.150000,0.0,0.0,1.395776e-10,1.473549,1.154765e-08,1.926451,67.878788
97,0.150000,6.177886e-10,2.921987,1.808047e-10,13.073549,3.981066e-10,2.411965e-10,0.150000,12.078013,9.133492e-10,4.322243e-10,0.150000,0.0,0.0,7.685422e-11,1.473549,9.133078e-09,1.926451,68.585859
98,0.150000,4.561970e-10,2.921987,1.615880e-10,13.073549,1.758208e-10,1.655109e-10,0.150000,12.078013,1.020553e-09,4.078877e-10,0.150000,0.0,0.0,2.200688e-11,1.473549,7.276758e-09,1.926451,69.292929


In [ ]:
re_atp

## Try with just enzyme 1 reactions

In [48]:
E1 = Enzyme(enzyme_name = "enzyme1", substrate = ["glucose"], 
            fuel = ['atp', 'atp'],product = ['f16p'], waste = ['adp', 'adp']) #, parameters = parameters)
    

myMixture = EnergyTxTl(components = [E1])
CRN = myMixture.compile_crn()

print(CRN.pretty_print(show_rates = False, show_attributes = False, show_materials = False))


comp2 [enzyme_enzyme1, molecule_f16p, metabolite_adp, metabolite_adp]
Species (7) = {0. enzyme[enzyme1], 1. metabolite[atp], 2. molecule[glucose], 3. molecule[f16p], 4. metabolite[adp], 5. complex[enzyme[enzyme1]:2x_metabolite[atp]:molecule[glucose]], 6. complex[enzyme[enzyme1]:2x_metabolite[adp]:molecule[f16p]]}
Reactions (3) = [
0. enzyme[enzyme1] + 2 metabolite[atp] + molecule[glucose] <--> complex[enzyme[enzyme1]:2x_metabolite[atp]:molecule[glucose]]        
1. complex[enzyme[enzyme1]:2x_metabolite[atp]:molecule[glucose]] --> complex[enzyme[enzyme1]:2x_metabolite[adp]:molecule[f16p]]        
2. complex[enzyme[enzyme1]:2x_metabolite[adp]:molecule[f16p]] <--> enzyme[enzyme1] + molecule[f16p] + 2 metabolite[adp]        
]


/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/biocrnpyler-0.1-py3.7.egg/biocrnpyler/component.py:11: UserWarning: get_species() not defined for component {self.name}, None returned.


In [41]:
CRN.write_sbml_file("CRN.sbml")
timepoints = np.linspace(0,70,100)
e = 0.15
atp = 15
x0 = {'molecule_glucose':15,
      'metabolite_atp': atp,
      "enzyme_enzyme1":e,}


re, me = CRN.simulate_with_bioscrape_via_sbml(timepoints, initial_condition_dict = x0, file = 'CRN.sbml')



In [42]:
#colors = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3','#a6d854','#ffd92f']
colors=['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02', '#a6761d']
p1 = bokeh.plotting.figure(width =450, height = 250, title = 'Dynamics rxn 1,2,3', x_axis_label = 'time',
                          y_axis_label = 'concentration')
p1.line(timepoints, re['metabolite_atp'], legend_label = 'atp', line_width = 2, color = colors[0])
p1.line(timepoints, re['molecule_glucose'], legend_label = 'glucose', line_width = 2, color = colors[1])
p1.line(timepoints, re['enzyme_enzyme1'], legend_label = 'enzyme1', line_width = 2, color = colors[2])
p1.line(timepoints, re['metabolite_adp'], legend_label = 'adp', line_width = 2, color = colors[3])
p1.line(timepoints, re['molecule_f16p'], legend_label = 'f16p', line_width = 2, color = colors[4])


p1.legend.click_policy="hide"

bokeh.io.show(row(p1))

<font color = purple>
    it seems that once enzyme1 is used is it never recycled
    </font>

In [33]:
re

,enzyme_enzyme1,metabolite_atp,molecule_glucose,molecule_f16p,metabolite_adp,complex_enzyme_enzyme1_2x_metabolite_atp_molecule_glucose,complex_enzyme_enzyme1_2x_metabolite_adp_molecule_f16p,time
0,150.000000,1.500000e+01,15.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000
1,143.300575,1.207394e-07,8.263585,0.036990,0.036990,2.858921e-03,0.127596,0.707071
2,143.277933,4.603744e-11,8.263876,0.014057,0.014057,1.089969e-06,0.048051,1.414141
3,143.272497,1.890294e-14,8.263876,0.008620,0.008620,4.475310e-10,0.029464,2.121212
4,143.270092,4.460170e-17,8.263876,0.006216,0.006216,1.048587e-12,0.021246,2.828283
...,...,...,...,...,...,...,...,...
95,143.264112,3.038223e-22,8.263876,0.000236,0.000236,7.192349e-18,0.000805,67.171717
96,143.264110,8.338424e-22,8.263876,0.000233,0.000233,1.973997e-17,0.000797,67.878788
97,143.264107,1.064764e-21,8.263876,0.000231,0.000231,2.520683e-17,0.000789,68.585859
98,143.264105,6.187247e-22,8.263876,0.000228,0.000228,1.464746e-17,0.000781,69.292929
